In [59]:
!pip install mip

In [60]:
from queue import Queue

In [61]:
from mip import *

In [104]:
# Função para ler dados do problema de um arquivo
def read(filename):
    with open(filename, 'r') as file:
        lista = file.readlines()  # Lê todas as linhas do arquivo
        lista = [x.strip().split() for x in lista]  # Remove espaços em branco e quebra cada linha em lista
        lista = [[int(y) for y in x] for x in lista]  # Converte todos os elementos para inteiros
        variaveis = lista[0][0]  # Obtém o número de variáveis do problema
        num_restricoes = lista[0][1]  # Obtém o número de restrições do problema
        objetivo = lista[1]  # Obtém os coeficientes da função objetivo
        restricoes = lista[2:]  # Obtém as restrições do problema
    return variaveis, num_restricoes, objetivo, restricoes

# Função para resolver a relaxação linear de um modelo
def solve_relaxation(model, restricoes, objetivo, variaveis):
    # Define a função objetivo do modelo usando a soma ponderada
    model.objective = xsum(objetivo[i] * model.vars[i] for i in range(variaveis))

    # Adiciona as restrições ao modelo
    for data in restricoes:
        model += xsum(data[i] * model.vars[i] for i in range(variaveis)) <= data[-1]

    model.optimize()

    # Se o modelo tem solução, retorna o valor objetivo e a solução; senão, retorna None
    if model.num_solutions:
        return model.objective_value, [v.x for v in model.vars]
    else:
        return None, None

# Função para determinar a variável de ramificação baseada na casa decimal mais próxima de 0.5
def branch(solucao, alpha=0.0001):
    # Filtra as variáveis que não são inteiras dentro de uma tolerância alpa
    nao_binarias = [(i, abs(x - round(x))) for i, x in enumerate(solucao) if abs(x - round(x)) > alpha]
    if not nao_binarias:
        return None

    # Ordena as variáveis não binárias pela distância da parte decimal a 0.5
    nao_binarias.sort(key=lambda x: -abs(x[1] - 0.5))
    branchV = nao_binarias[0][0]  # Escolhe a variável com a casa decimal mais próxima de 0.5
    return branchV

# Função principal do algoritmo branch-and-bound
def branch_and_bound(filename):
    variaveis, num_restricoes, objetivo, restricoes = read(filename)
    # Cria uma fila para gerenciar os nós do branch-and-bound
    queue = Queue()
    queue.put(([], {i: None for i in range(variaveis)}))  # Coloca o estado inicial na fila

    best_solution = None  # Inicializa a melhor solução como None
    best_objective = float('-inf')  # Inicializa o melhor valor objetivo como infinito negativo

    # Enquanto houver elementos na fila, processa cada um
    while not queue.empty():
        restricao_atual, integridade_atual = queue.get()  # Obtém o próximo elemento da fila

        m = Model(sense=MAXIMIZE)  # Cria um novo modelo de maximização
        # Define as variáveis do modelo como contínuas com limites entre 0 e 1
        m.vars = [m.add_var(var_type=CONTINUOUS, lb=0, ub=1) for i in range(variaveis)]

        # Adiciona as restrições atuais ao modelo
        for (x, y) in restricao_atual:
            if y is not None:
                m += m.vars[x] == y

        # Resolve a relaxação linear do modelo atual
        objective_value, solution = solve_relaxation(m, restricoes, objetivo, variaveis)

        # Se o valor objetivo da relaxação linear é pior que o melhor encontrado, ignora o ramo
        if objective_value is None or objective_value <= best_objective:
            continue

        # Se a solução é viável (todas as variáveis são inteiras), atualiza a melhor solução
        if all(x in (0, 1) for x in solution):
            if objective_value > best_objective:
                best_solution = solution
                best_objective = objective_value
            continue

        # Determina a próxima variável para ramificação
        branch_var = branch(solution)
        if branch_var is not None:
            # Ramifica nas variáveis, adicionando restrições e colocando novos nós na fila
            for i in [0, 1]:
                nova_restricao = restricao_atual + [(branch_var, i)]
                queue.put((nova_restricao, integridade_atual))

    # Retorna o melhor valor objetivo e a melhor solução encontrados
    return best_objective, best_solution

In [105]:
optimal_value, optimal_solucao = branch_and_bound('teste1.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 20.0
Optimal solucao: [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]


In [94]:
optimal_value, optimal_solucao = branch_and_bound('teste2.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 24.0
Optimal solution: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]


In [95]:
optimal_value, optimal_solucao = branch_and_bound('teste3.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 19.0
Optimal solution: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]


In [96]:
optimal_value, optimal_solucao = branch_and_bound('teste4.txt')
print("Optimal value:", optimal_value)
print("Optimal solucao:", optimal_solucao)

Optimal value: 10.0
Optimal solution: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
